In [141]:
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import time
import matplotlib.pyplot as plt
import numpy as np

In [142]:
os.listdir("../../data/static_files/")

['LargeCapCD.xlsx',
 'LargeCapComms.xlsx',
 'LargeCapCS.xlsx',
 'LargeCapEner.xlsx',
 'LargeCapFina.xlsx',
 'LargeCapHC.xlsx',
 'LargeCapInd.xlsx',
 'LargeCapIT.xlsx',
 'LargeCapMat.xlsx',
 'LargeCapRE.xlsx',
 'LargeCapUtil.xlsx',
 'nasdaq_nyse.xlsx']

In [143]:
load_dotenv("../../.env",override=True)
consumerKey = os.environ.get("CONSUMER_KEY")


In [144]:
pd.set_option("display.max_columns",500)
pd.set_option("display.max_rows",10)

In [145]:
def request_fundamentals(Tickers: list, consumerKey: str, how: str = 'DF') -> pd.DataFrame:
    """_summary_

    Args:
        Tickers (list)
        consumerKey (str)

    Returns:
        pd.DataFrame
    """
    symbolList = ",".join([i for i in Tickers])
    endpoint = f'''https://api.tdameritrade.com/v1/instruments?
        &symbol={symbolList}&projection=fundamental'''
    page = requests.get(url=endpoint, 
        params={'apikey' : consumerKey})
    content = json.loads(page.content)

    allDict = {}
    for k in list(content.keys()):
        allDict[k] = content[k]['fundamental']
    fundamentals = pd.DataFrame(allDict).T
    
    if how=='DF':
        return fundamentals
    else:
        return allDict

def request_quotes(consumerKey: str, Tickers: list) -> pd.DataFrame:
    """_summary_

    Args:
        consumerKey (str): _description_
        Tickers (list): _description_

    Returns:
        pd.DataFrame: _description_
    """
    
    symbolList = ",".join([i for i in Tickers])
    endpoint = f"https://api.tdameritrade.com/v1/marketdata/quotes?symbol={symbolList}"
    page = requests.get(url=endpoint, 
                params={'apikey' : consumerKey})
    content = json.loads(page.content)
    content = pd.DataFrame(content).T
    return content

In [146]:
suffixes = ['CD','Comms','CS','Ener','Fina','HC','Ind','IT','Mat','RE','Util']

In [147]:
Industry = suffixes[7]
symbols = pd.read_excel(f"../../data/static_files/LargeCap{Industry}.xlsx",header=1)
symbols['Symbol'].fillna("-",inplace=True)
Tickers = list(symbols[(~symbols['Symbol'].astype(str).str.contains('-'))]['Symbol'])


c:\Users\jliv\Desktop\research\python_research\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [148]:

FDF = request_fundamentals(Tickers=Tickers,consumerKey=consumerKey)
Quotes = request_quotes(Tickers=Tickers,consumerKey=consumerKey)

In [149]:
highGood = ['dividendAmount','dividendYield',
    'grossMarginTTM','grossMarginMRQ','netProfitMarginTTM','netProfitMarginMRQ',
    'operatingMarginTTM','operatingMarginMRQ',
    'returnOnEquity', 'returnOnAssets', 'returnOnInvestment',
    'quickRatio','currentRatio','interestCoverage',
    'epsTTM','epsChangePercentTTM','epsChangeYear','epsChange',
    'revChangeYear', 'revChangeTTM','revChangeIn','marketCap',
    'divGrowthRate3Year','dividendPayAmount','beta']

lowGood = [
    'peRatio', 'pegRatio', 'pbRatio', 'prRatio', 'pcfRatio',
    'totalDebtToCapital', 'ltDebtToEquity', 'totalDebtToEquity',
    'shortIntToFloat', 'shortIntDayToCover'
]

In [150]:
dat = Quotes[['symbol','description','lastPrice',
    'volatility','peRatio','divAmount','divYield',
    ]].merge(FDF[['symbol']+[
        i for i in FDF.columns if i not in Quotes.columns
    ]],on='symbol',how='inner')

dat=pd.DataFrame(np.where(dat==0,np.nan,dat),columns = dat.columns)

In [151]:
def MinMaxScaler(x: pd.Series) -> pd.Series:
    """_summary_

    Args:
        x (pd.Series): _description_

    Returns:
        pd.Series: _description_
    """
    return (x-np.nanmin(x))/(np.nanmax(x)-np.nanmin(x))

def MaxMinScaler(x: pd.Series) -> pd.Series:
    """_summary_

    Args:
        x (pd.Series): _description_

    Returns:
        pd.Series: _description_
    """
    return 1-MinMaxScaler(x)

In [152]:
rankdat = dat.copy()

In [153]:
metricsFamilies = {
    'debt': {'columns': ['totalDebtToCapital','ltDebtToEquity','totalDebtToEquity'],
            'ascending':True},
    'ratio': {'columns': ['quickRatio','currentRatio','interestCoverage'],
            'ascending':False},
    'change': {'columns': ['epsChangePercentTTM',
       'epsChangeYear', 'epsChange', 'revChangeYear', 'revChangeTTM',
       'revChangeIn'],
            'ascending':False},
    'profit': {'columns': ['epsTTM','grossMarginTTM', 'grossMarginMRQ', 'netProfitMarginTTM',
       'netProfitMarginMRQ', 'operatingMarginTTM', 'operatingMarginMRQ'],
            'ascending':False},
    'value': {'columns': ['peRatio','pegRatio', 'pbRatio', 'prRatio',
       'pcfRatio'],
            'ascending':True},
    'return': {'columns':  ['returnOnEquity', 'returnOnAssets', 'returnOnInvestment'],
            'ascending':False},
}

In [154]:
def convert_to_ranking(rankdat: pd.DataFrame, 
    metricsFamilies: dict[str,dict]) -> pd.DataFrame:
    """_summary_

    Args:
        rankdat (pd.DataFrame): _description_
        metricsFamilies (dict[str,dict]): _description_

    Returns:
        pd.DataFrame: _description_
    """
    families = list(metricsFamilies.keys())
    
    for f in families:
        columns = metricsFamilies[f]['columns']
        ascending= metricsFamilies[f]['ascending']
        for v in columns:
            rankdat[v] = rankdat[v].rank(ascending=ascending)

        rankdat[f'{f}Known']=np.sum(np.where(rankdat[columns].isna(),0,1),axis=1)

        rankdat[columns]=rankdat[columns]/np.nanmax(rankdat[columns],axis=0)

    return rankdat


In [155]:
rankdat = convert_to_ranking(rankdat,metricsFamilies)

C:\Users\jliv\AppData\Local\Temp\ipykernel_14784\933486261.py:22: RuntimeWarning: All-NaN axis encountered
  rankdat[columns]=rankdat[columns]/np.nanmax(rankdat[columns],axis=0)


In [157]:
rankedCols = []
for k in metricsFamilies.keys():
    rankedCols=rankedCols+metricsFamilies[k]['columns']
    
knownCols = [i for i in rankdat.columns if 'Known' in i]
rankdat['metricsInformed'] = np.sum(rankdat[knownCols],axis=1)


In [158]:
rankdat['meanRank']=np.sum(rankdat[rankedCols],axis=1)/rankdat['metricsInformed']

In [159]:
out=rankdat[['symbol','description','meanRank','metricsInformed']].merge(
    dat[['symbol']+rankedCols],
    on=['symbol'],
    how='left'
)

In [160]:
out.sort_values(by='meanRank').head(10)

,symbol,description,meanRank,metricsInformed,totalDebtToCapital,ltDebtToEquity,totalDebtToEquity,quickRatio,currentRatio,interestCoverage,epsChangePercentTTM,epsChangeYear,epsChange,revChangeYear,revChangeTTM,revChangeIn,epsTTM,grossMarginTTM,grossMarginMRQ,netProfitMarginTTM,netProfitMarginMRQ,operatingMarginTTM,operatingMarginMRQ,peRatio,pegRatio,pbRatio,prRatio,pcfRatio,returnOnEquity,returnOnAssets,returnOnInvestment
115,PATH,"UiPath, Inc. Class A Common Stock",0.233056,6,NaN,NaN,NaN,NaN,4.58293,NaN,NaN,NaN,NaN,NaN,35.40498,NaN,NaN,82.6219,81.78572,NaN,NaN,NaN,NaN,NaN,NaN,3.49061,6.4721,NaN,NaN,NaN,NaN
146,TER,"Teradyne, Inc. - Common Stock",0.247321,20,3.13642,2.81894,3.23798,2.47661,2.84735,NaN,NaN,NaN,NaN,NaN,NaN,11.30519,4.74696,60.01053,60.22948,24.49157,23.52462,28.82248,29.49572,15.3319,NaN,5.30732,3.5302,12.68109,34.92339,22.44449,29.45617
150,TWLO,Twilio Inc. Class A Common Stock,0.259694,10,8.58174,9.27462,9.38733,NaN,6.41803,NaN,NaN,NaN,NaN,NaN,50.85012,7.76718,NaN,47.96285,47.20275,NaN,NaN,NaN,NaN,NaN,NaN,1.27413,4.06181,NaN,NaN,NaN,NaN
69,HCP,"HashiCorp, Inc. - Class A Common Stock",0.277796,7,NaN,NaN,NaN,NaN,4.71571,NaN,NaN,NaN,NaN,NaN,51.88103,12.85073,NaN,79.9814,81.08253,NaN,NaN,NaN,NaN,NaN,NaN,4.63139,14.48422,NaN,NaN,NaN,NaN
149,TSM,Taiwan Semiconductor Manufacturing Company Ltd.,0.284357,24,26.70323,31.23432,36.63757,1.99757,2.25482,NaN,37.56191,76.42164,NaN,NaN,29.29828,8.76951,4.5822,54.96044,59.06083,40.62169,44.40407,44.70883,49.07182,13.99,0.370389,4.24822,5.37493,8.37775,33.96735,20.51166,25.73269
155,UMC,United Microelectronics Corporation (NEW) Comm...,0.287879,24,20.44695,18.06314,25.72732,2.40852,2.68129,NaN,73.19796,87.62883,NaN,NaN,26.25198,7.31516,0.82099,37.96148,43.36661,28.46849,31.63786,28.95249,35.21475,6.8,0.103837,1.65464,2.12425,4.3554,23.62713,14.88621,18.51822
99,MU,"Micron Technology, Inc. - Common Stock",0.290371,22,12.15567,13.63135,13.83774,2.00531,2.88911,NaN,50.64456,NaN,NaN,NaN,11.01967,NaN,7.7337,45.18499,39.47012,28.23005,22.535,31.27316,22.89628,6.7949,0.143105,1.22866,1.98114,3.85694,18.51449,13.87814,15.62126
139,STM,STMicroelectronics N.V. Common Stock,0.306529,24,19.77082,23.48134,24.80063,1.65679,2.27829,NaN,76.0977,106.1272,NaN,NaN,18.70014,8.20643,3.03323,45.36644,47.40683,20.1047,22.6479,23.84692,26.16628,10.3,0.14431,2.97653,2.10106,7.52676,30.1616,17.33402,22.31032
151,TXN,Texas Instruments Incorporated - Common Stock,0.311894,25,33.95201,47.86404,51.40505,4.23594,5.09492,56.91837,27.53735,19.44764,NaN,NaN,16.88343,6.25892,9.14467,69.25786,69.55104,43.77807,43.95625,51.52103,52.24482,16.5681,0.634263,10.37069,7.44882,15.36668,67.65696,37.84583,42.28769
118,PCOR,"Procore Technologies, Inc. Common Stock",0.315666,10,3.89544,4.05333,4.05333,NaN,1.70697,NaN,NaN,52.90595,NaN,NaN,NaN,7.95469,NaN,79.90748,78.66787,NaN,NaN,NaN,NaN,NaN,NaN,6.25204,11.99129,NaN,NaN,NaN,NaN
